# Rustyread analysis

## Compare read generation

Run `snakemake --use-conda -j {numbre of job} {your option} data/compare/rustyread.paf data/compare/badread.paf data/real_read.paf` before run next cells

### Read length

In [ ]:
import pandas
import altair

altair.data_transformers.disable_max_rows()

def get_length(path):
    with open(path) as fh:
        line_count = 0
        for line in fh:
            line_count += 1
            if line_count % 2 == 0:
                yield len(line)

data = [('real', x) for x in get_length("data/real_reads.fastq")]
data += [('rustyread', x) for x in get_length("data/compare/rustyread.fastq")]
#data += [('badread', x) for x in get_length("data/compare/rustyread.fastq")]

df = pandas.DataFrame(data, columns=["origin", "length"])

print(df.groupby("origin").count())
print(df.groupby("origin").mean())
print(df.groupby("origin").std())

altair.Chart(df).mark_bar().encode(
    x=altair.X("length", bin=altair.Bin(maxbins=200)),
    y='count()',
    color='origin',
).interactive()

### Read identity

In [ ]:
import pandas
import altair

altair.data_transformers.disable_max_rows()

def get_identity(path):
    with open(path) as fh:
        for line in fh:
            line = line.split("\t")
            nm = int([e.split(":")[2] for e in line if e.startswith("NM:i:")][0])
            yield 1 - nm / int(line[1])

data = [('real', x) for x in get_identity("data/real_reads.paf")]
data += [('rustyread', x) for x in get_identity("data/compare/rustyread.paf")]
#data += [('badread', x) for x in get_identity("data/compare/badread.paf")]

df = pandas.DataFrame(data, columns=["origin", "identity"])

altair.Chart(df).mark_bar().encode(
    x=altair.X("identity", bin=altair.Bin(maxbins=200)),
    y='count()',
    color='origin',
).interactive()

### Quality

In [ ]:
import pandas
import altair
from collections import Counter

altair.data_transformers.disable_max_rows()

def get_qual(path):
    with open(path) as fh:
        line_count = 0
        for line in fh:
            line_count += 1
            if line_count % 4 == 0:
                for q in line:
                    yield ord(q) - 33
    
tmp = Counter(get_qual("data/real_reads.fastq"))
data = [("real", i, tmp[i]) for i in sorted(tmp.keys()) if i > 0]

tmp = Counter(get_qual("data/compare/rustyread.fastq"))
data += [("rustyread", i, tmp[i]) for i in sorted(tmp.keys()) if i > 0]

#tmp = Counter(get_qual("data/compare/badread.fastq"))
#data += [("badread", i, tmp[i]) for i in sorted(tmp.keys()) if i > 0]

df = pandas.DataFrame(data, columns=["origin", "x", "y"])

altair.Chart(df).mark_bar().encode(
    x='x',
    y='y',
    color='origin',
)

## Thread scalability

Run `snakemake --use-conda -j 1 {your option} -R ms_all` before run next cell

In [ ]:
import os
import re
import altair
import pandas

data = list()
with os.scandir("benchmarks/ms/") as it:
    for entry in it:
        if entry.is_file() and entry.name.startswith("rustyread_t"):
            threads = int(re.search("rustyread_t(\d+).tsv", entry.name).group(1))
            with open(entry.path) as fh:
                next(fh)
                for line in fh:
                    wall_time = float(line.split("\t")[0])
                    data.append((threads, wall_time))
        
raw_df = pandas.DataFrame(data, columns=["nb_threads", "wall_time"])

df = pandas.DataFrame(list(set(raw_df["nb_threads"])), columns=["nb_threads"])
df["mean"] = list(raw_df.groupby("nb_threads").mean()["wall_time"])
df["std"] = list(raw_df.groupby("nb_threads").std()["wall_time"])

df["linear"] = [df["mean"][0] / x for x in df["nb_threads"]]

base = altair.Chart(df).transform_calculate(
    ymin="datum.mean-datum.std",
    ymax="datum.mean+datum.std"
)

point = base.mark_circle().encode(
    x='nb_threads',
    y='mean',
)

line = base.mark_line().encode(
    x='nb_threads',
    y='linear'
)

errorbars = base.mark_errorbar().encode(
    x='nb_threads',
    y='ymin:Q',
    y2='ymax:Q'
)

point + errorbars + line